In [1]:
# TensorFlow 라이브러리를 사용한 컨볼루션 신경망 구현 예.
# 이 예제는 손으로 쓴 숫자들을 모와 둔 MNIST 데이터베이스를 사용하고 있습니다.
# (http://yann.lecun.com/exdb/mnist/)

# 작성자: Aymeric Damien
# 수정자: 정 경태
# Project: https://github.com/aymericdamien/TensorFlow-Examples/

In [2]:
import tensorflow as tf

# MINST 데이터 가져오기
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# 파라메터(Parameters)
learning_rate = 0.001
training_iters = 1000
batch_size = 100
display_step = 10

# 네트워크 파라메터(Parameters)
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf 그래프(Graph) 입력
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

# 이미지 출력
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, 100)

In [4]:
# 간편 레퍼(wrapper) 만들기
def conv2d(x, W, b, strides=1):
    # 바이어스 및 RELU 활성화 Conv2D 래퍼
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    # MaxPool2D 래퍼
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

# 모델 생성
def conv_net(x, weights, biases, dropout):
    # 입력 이미지 리쉐입(Reshape)
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution 층
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution 층
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected 층
    # conv2 출력을 fully connected 층의 입력에 맞도록 변형
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Dropout 적용
    fc1 = tf.nn.dropout(fc1, dropout)

    # 출력 클래스 예측
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [5]:
# 가중치 및 바이어스 저장층
weights = {
    # 5x5 conv, 1 입력, 32 출력
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 입력, 64 출력
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 입력, 1024 출력
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 입력, 10 출력 (클래스 예측)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

tf.summary.histogram("weights_wc1", weights['wc1'])
tf.summary.histogram("weights_wc2", weights['wc2'])
tf.summary.histogram("weights_wd1", weights['wd1'])
tf.summary.histogram("weights_out", weights['out'])

tf.summary.histogram("biases_bc1", biases['bc1'])
tf.summary.histogram("biases_bc2", biases['bc2'])
tf.summary.histogram("biases_bd1", biases['bd1'])
tf.summary.histogram("biases_out", biases['out'])

# 모델 생성
pred = conv_net(x, weights, biases, keep_prob)

# 손실 및 최적화 정의
with tf.name_scope("cost"):
    diff = tf.nn.softmax_cross_entropy_with_logits(pred, y)
    with tf.name_scope('total'):
        cost = tf.reduce_mean(diff)
    tf.summary.scalar('cost', cost)
    
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    

# 평가 모델
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

# summary 통합
merged = tf.summary.merge_all()

# 변수 초기화
init = tf.global_variables_initializer()

In [6]:
# 세션 설정
session_conf = tf.ConfigProto()
session_conf.gpu_options.allow_growth = True

# 그래프 실행
with tf.Session(config=session_conf) as sess:
    sess.run(init)
    
    # writer 생성
    train_writer = tf.summary.FileWriter("./board/sample_03/train", sess.graph)
    test_writer = tf.summary.FileWriter("./board/sample_03/test")
    
    # 최대 반복횟수에 도달할 때까지 훈련 함
    for i in range(training_iters):
        if i % 10 == 0:  # Record summaries and test-set accuracy
            summary, acc = sess.run([merged, accuracy],
                                    feed_dict={x: mnist.test.images[:256],
                                          y: mnist.test.labels[:256],
                                          keep_prob: 1.})
            test_writer.add_summary(summary, i)
            print('Accuracy at step %s: %s' % (i, acc))
        else:  # Record train set summaries, and train
            batch_x, batch_y = mnist.train.next_batch(batch_size)

            if i % 100 == 99:  # Record execution stats
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
                summary, _ = sess.run([merged, train_step],
                                  feed_dict={x: batch_x, y: batch_y, keep_prob: dropout},
                                  options=run_options,
                                  run_metadata=run_metadata)
                train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
                train_writer.add_summary(summary, i)
                print('Adding run metadata for', i)
            else:  # Record a summary
                summary, _ = sess.run([merged, train_step], feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
                train_writer.add_summary(summary, i)
    
    train_writer.close()
    test_writer.close()
    print("최적화 종료됨!")
    

    # 256개의 mnist 테스트 이미지 정확도 계산
#     print("Testing Accuracy:", \
#         sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
#                                       y: mnist.test.labels[:256],
#                                       keep_prob: 1.}))

Accuracy at step 0: 0.0859375
Accuracy at step 10: 0.257812
Accuracy at step 20: 0.460938
Accuracy at step 30: 0.632812
Accuracy at step 40: 0.761719
Accuracy at step 50: 0.804688
Accuracy at step 60: 0.8125
Accuracy at step 70: 0.839844
Accuracy at step 80: 0.867188
Accuracy at step 90: 0.871094
Adding run metadata for 99
Accuracy at step 100: 0.882812
Accuracy at step 110: 0.894531
Accuracy at step 120: 0.898438
Accuracy at step 130: 0.914062
Accuracy at step 140: 0.90625
Accuracy at step 150: 0.917969
Accuracy at step 160: 0.925781
Accuracy at step 170: 0.921875
Accuracy at step 180: 0.929688
Accuracy at step 190: 0.929688
Adding run metadata for 199
Accuracy at step 200: 0.925781
Accuracy at step 210: 0.957031
Accuracy at step 220: 0.960938
Accuracy at step 230: 0.953125
Accuracy at step 240: 0.957031
Accuracy at step 250: 0.964844
Accuracy at step 260: 0.960938
Accuracy at step 270: 0.960938
Accuracy at step 280: 0.964844
Accuracy at step 290: 0.96875
Adding run metadata for 299
A